In [64]:
import cv2
import numpy as np

In [69]:
img = cv2.imread('bg_11.jpg')

In [71]:


def color_balance(img):
    # 1. Calculate the average intensity of each color channel
    red_avg = np.mean(img[:,:,2])
    green_avg = np.mean(img[:,:,1])
    blue_avg = np.mean(img[:,:,0])

    # 2. Compute the difference between the average intensity and the mean intensity
    mean_intensity = np.mean([red_avg, green_avg, blue_avg])
    red_diff = mean_intensity - red_avg
    green_diff = mean_intensity - green_avg
    blue_diff = mean_intensity - blue_avg

    # 3. Adjust the intensity of each channel
    img[:,:,2] = np.clip(img[:,:,2] + red_diff, 0, 255)
    img[:,:,1] = np.clip(img[:,:,1] + green_diff, 0, 255)
    img[:,:,0] = np.clip(img[:,:,0] + blue_diff, 0, 255)

    # 4. Return the processed image
    return img



# Apply color balance
balanced_img = color_balance(img)

# Save the result
cv2.imwrite('balanced.jpg', balanced_img)

True

In [72]:
import cv2
import numpy as np

def estimate_background_light_for_each_channel(image):
    # Initialize background light for each channel
    background_light_channels = np.zeros(3)
    
    # Convert image to grayscale
    gray_image = image
    
    # Initialize threshold for quadtree division
    threshold = gray_image.size * 0.001
    
    # Define function for quadtree division
    def quadtree_division(image, threshold):
        variance = np.var(image)
        
        # If variance is less than threshold, return mean intensity as background light
        if variance < threshold:
            return np.mean(image)
        
        # Otherwise, divide image into four quadrants
        height, width = image.shape
        half_height, half_width = height // 2, width // 2
        quadrants = [
            image[:half_height, :half_width],
            image[:half_height, half_width:],
            image[half_height:, :half_width],
            image[half_height:, half_width:]
        ]
        
        # Recursively call quadtree_division on each quadrant
        background_lights = [quadtree_division(quadrant, threshold) for quadrant in quadrants]
        
        # Return the minimum background light among quadrants
        return min(background_lights)
    
    # Start quadtree division for each channel
    for i in range(3):  # Assuming BGR image
        background_light_channels[i] = quadtree_division(image[:,:,i], threshold)
    
    return background_light_channels

# Example usage:
# Read RGB image
image = cv2.imread('balanced.jpg')

# Estimate background light for each channel
background_light_channels = estimate_background_light_for_each_channel(image)
print("Estimated Background Light for Each Channel:", background_light_channels)


Estimated Background Light for Each Channel: [107.00965833 106.2381     106.08169167]


In [74]:
def calculate_contrast_and_complex_performance_index(I, t):
    
    # Calculate the number of channels
    num_channels = I.shape[2]
    
    # Initialize arrays to store contrast values Ci for each channel
    Ci = np.zeros(num_channels)
    
    # Loop through each color channel
    for i in range(num_channels):
       
        # Calculate the contrast value Ci for the current channel
        squared_diff_channel = (I[:,:,i] - np.mean(I[:,:,i])) ** 2
        Ci[i] = np.sum(squared_diff_channel / (t ** 2 * I.size))
    
    # Calculate the complex contrast performance index Ec
    Ec = -np.sum(Ci)
    
    return Ci, Ec



In [76]:
def calculate_information_loss_El(I, A, t):
    # Initialize J(x) array
    J = np.zeros_like(I)
    
    # Calculate J(x) using the restoration model equation for each channel
    for i in range(3):  # Assuming RGB image

        J[:,:,i] = (1 / t) * (I[:,:,i] - A[i]) + A[i]
    
    # Ensure J is within the allowable range (0 to 255)
    J_clipped = np.clip(J, 0, 255)
    
    # Calculate information loss function El
    El = 0
    
    # Loop through each color channel
    for i in range(3):  # Assuming RGB image
        # Calculate information loss for each pixel in the channel
        loss_channel = (np.minimum(0, J_clipped[:,:,i])**2) + np.maximum(0, J_clipped[:,:,i] - 255)
        
        # Sum up the information loss for the channel
        El += np.sum(loss_channel)
    
    return El



In [77]:
from scipy.optimize import minimize





def combined_objective_function(Ec, El, lambda_L):
    # Calculate the combined objective function
    return Ec + lambda_L * El

def optimize_transmittance(I, A, lambda_L, initial_guesses):
    # Initialize lists to store results
    optimal_t = []
    optimal_Ec = []
    optimal_El = []
    optimal_E = []
    print(len(initial_guesses),"kk")
    # Loop through each initial guess for t
    for i, initial_guess in enumerate(initial_guesses):
       

       
            # Calculate contrast value Ci and complex contrast performance index Ec
        Ci, Ec = calculate_contrast_and_complex_performance_index(I, initial_guess)
    
            # Calculate information loss El
        El = calculate_information_loss_El(I, A, initial_guess)
            
            # Calculate combined objective function E
        E = combined_objective_function(Ec, El, lambda_L)
            
            # Store results
        
                
        
        
        # Store the results for this initial guess
        optimal_t.append(initial_guess)
        optimal_Ec.append(Ec)
        optimal_El.append(El)
        optimal_E.append(E)
    
    # Find the index corresponding to the minimum value of E among all initial guesses
    best_index = np.argmin(optimal_E)
    
    # Return the best results
    return optimal_t[best_index], optimal_Ec[best_index], optimal_El[best_index], optimal_E[best_index]

# Example usage:
# Read RGB image
I = cv2.imread('balanced.jpg')

# Background light A obtained from Step 1 (assuming some value)
A = background_light_channels

# Tradeoff factor
lambda_L = 5

# Initial guesses for transmittance
constant = 0.1  # or any small value you prefer
initial_guesses = [np.around(np.clip(np.random.rand(*I.shape[:2]) + constant, constant,1), decimals=4) for _ in range(1000)]

# Optimize transmittance
t_optimal, Ec_optimal, El_optimal, E_optimal = optimize_transmittance(I, A, lambda_L,initial_guesses )

# Print the results
print("Optimized Transmittance (t*):", t_optimal)
print("Optimal Ec:", Ec_optimal)
print("Optimal El:", El_optimal)
print("Optimal E:", E_optimal)


1000 kk
Optimized Transmittance (t*): [[0.3141 0.2741 1.     ... 0.5924 0.8642 0.4519]
 [0.4508 0.8776 1.     ... 0.1472 0.7002 1.    ]
 [0.8963 0.189  1.     ... 0.2474 0.5361 0.759 ]
 ...
 [0.8265 0.8611 0.7942 ... 0.2458 1.     0.6034]
 [0.6616 0.9934 0.7592 ... 0.8511 0.3587 1.    ]
 [1.     0.7182 0.4092 ... 0.1923 1.     0.8179]]
Optimal Ec: -2186.725305872255
Optimal El: 39321033
Optimal E: 196602978.27469411


In [78]:
import cv2
import numpy as np

def restore_image(I, A, t):
    # Initialize the restored image
    restored_image = np.zeros_like(I)

    # Iterate over each pixel in the image
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            # Apply the restoration model formula for each channel
            for k in range(3):  # Iterate over R, G, B channels
                restored_image[i, j, k] = (1 / t[i, j]) * (I[i, j, k] - A[k]) + A[k]

    return restored_image

# Example usage:
# Read RGB image
I = cv2.imread('balanced.jpg')

# Assuming background light A is already obtained
A = background_light_channels # Example background light (for each channel)


# Restore the image using the restoration model and the calculated transmittance map
restored_image = restore_image(I, A, t_optimal)
cv2.imwrite('restored_image_model.jpg', restored_image)
# Display or save the restored image
cv2.imshow('Restored Image', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [79]:
print(restored_image)

[[[240 229  99]
  [  4 247  98]
  [149 145 104]
  ...
  [191 173  68]
  [164 152  80]
  [217 194  57]]

 [[200 192 101]
  [154 150 103]
  [149 145 104]
  ...
  [190 120 213]
  [178 163  74]
  [157 146  84]]

 [[153 149 103]
  [ 73  55  95]
  [149 145 104]
  ...
  [ 53  10  16]
  [200 180  64]
  [172 158  76]]

 ...

 [[109 121 142]
  [109 121 140]
  [109 122 143]
  ...
  [ 41  93 117]
  [ 90 102 108]
  [ 77  97 107]]

 [[ 99 119 143]
  [102 116 132]
  [103 120 141]
  ...
  [ 91 105 113]
  [ 67 102 119]
  [ 92 104 110]]

 [[100 113 129]
  [ 97 115 137]
  [ 94 127 166]
  ...
  [ 44 110 142]
  [ 93 105 111]
  [ 89 104 112]]]


In [80]:
import cv2;
import math;
import numpy as np;

def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b);
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz);
    imvec = im.reshape(imsz,3);

    indices = darkvec.argsort();
    indices = indices[imsz-numpx::]

    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
       atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx;
    return A

def TransmissionEstimate(im,A,sz):
    omega = 0.95;
    im3 = np.empty(im.shape,im.dtype);

    for ind in range(0,3):
        im3[:,:,ind] = im[:,:,ind]/A[0,ind]

    transmission = 1 - omega*DarkChannel(im3,sz);
    return transmission

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r));
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r));
    cov_Ip = mean_Ip - mean_I*mean_p;

    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r));
    var_I   = mean_II - mean_I*mean_I;

    a = cov_Ip/(var_I + eps);
    b = mean_p - a*mean_I;

    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = mean_a*im + mean_b;
    return q;

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;
    eps = 0.0001;
    t = Guidedfilter(gray,et,r,eps);

    return t;

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype);
    t = cv2.max(t,tx);

    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]

    return res

if __name__ == '__main__':
    import sys
    try:
        fn = sys.argv[1]
    except:
        fn = './image/15.png'

    def nothing(*argv):
        pass

    src = cv2.imread("balanced.jpg");

    I = src.astype('float64')/255;
 
    dark = DarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(src,te);
    dcb = Recover(I,t,A,0.1);

  
    cv2.imshow('dcb',dcb);
    cv2.imwrite("dcbimage.png",J*255);
    cv2.waitKey();

    

In [82]:
import cv2
import numpy as np

def denormalize_image(image):
    # Denormalize image from range [0, 1] to [0, 255]
    image = image * 255

    # Convert image to uint8
    image = image.astype(np.uint8)

    return image
dcb=denormalize_image(dcb)

In [83]:
import numpy as np

def calculate_average_red_channel_value(image):
    # Assuming 'image' is a 3D NumPy array representing the image (height x width x channels)
    red_channel = image[:, :, 0]  # Assuming red channel is the first channel (index 0)
    average_red_value = np.mean(red_channel)
    return average_red_value

average_red = calculate_average_red_channel_value(dcb)
print("Average Red Channel Value:", average_red)

Average Red Channel Value: 82.602475


In [84]:
import numpy as np

def calculate_average_red_channel_value(image):
    # Assuming 'image' is a 3D NumPy array representing the image (height x width x channels)
    red_channel = image[:, :, 0]  # Assuming red channel is the first channel (index 0)
    average_red_value = np.mean(red_channel)
    return average_red_value

average_red = calculate_average_red_channel_value(restored_image)
print("Average Red Channel Value:", average_red)

Average Red Channel Value: 109.365625


In [99]:

def calculate_histogram_threshold(image):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    print(total_pixels)
    # Calculate histogram threshold (h1) as n*0.225%
    histogram_threshold = total_pixels * 0.00225
    
    return  histogram_threshold

histogram_threshold = calculate_histogram_threshold(restored_image)
print("Histogram Threshold:", histogram_threshold)


120000
Histogram Threshold: 270.0


In [100]:
r_threshold = 107
def calculate_histogram_threshold(image):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    print(total_pixels)
    # Calculate histogram threshold (h1) as n*0.225%
    histogram_threshold = total_pixels * 0.00225
    
    return  histogram_threshold

histogram_threshold = calculate_histogram_threshold(J)
print("Histogram Threshold:", histogram_threshold)


120000
Histogram Threshold: 270.0


In [101]:
import numpy as np

def histogram_stretching(image, R_ave, R_threshold):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    
    # Determine if attenuation of red light is slight or heavy
    if R_ave >= R_threshold:  # Slight attenuation
        channels = [0, 1, 2]  # Stretching for all channels (R, G, B)
    else:  # Heavy attenuation
        channels = [1, 2]  # Stretching for G, B channels only
    
    # Calculate satisfactory threshold ht as n * 0.225%
    ht = total_pixels * 0.00225
    
    # Calculate minimal and maximal scalar values based on ht
    imin = 0
    imax = 255
    
    # If ht is provided and it's greater than zero, update imin and imax
    if ht > 0:
        # Calculate lower and upper thresholds based on ht
        imin = np.min(image)
        imax = np.max(image)
        lower_threshold_count = int(ht)
        upper_threshold_count = int(total_pixels - ht)
        sorted_pixels = np.sort(image.flatten())
        imin = sorted_pixels[lower_threshold_count]
        imax = sorted_pixels[upper_threshold_count]

    # Apply histogram stretching to selected channels
    stretched_image = image.copy()
    for channel in channels:
        for i in range(height):
            for j in range(width):
                old_value = stretched_image[i, j, channel]
                if old_value < imin:
                    stretched_image[i, j, channel] = 0
                elif old_value > imax:
                    stretched_image[i, j, channel] = 255
                else:
                    stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)
    
    return stretched_image

# Example usage:
stretched_image = histogram_stretching(J, average_red, r_threshold)




In [102]:
import numpy as np

def histogram_stretching(image, R_ave, R_threshold):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    
    # Determine if attenuation of red light is slight or heavy
    if R_ave >= R_threshold:  # Slight attenuation
        channels = [0, 1, 2]  # Stretching for all channels (R, G, B)
    else:  # Heavy attenuation
        channels = [1, 2]  # Stretching for G, B channels only
    
    # Calculate satisfactory threshold ht as n * 0.225%
    ht = total_pixels * 0.00225
    
    # Calculate minimal and maximal scalar values based on ht
    imin = 0
    imax = 255
    
    # If ht is provided and it's greater than zero, update imin and imax
    if ht > 0:
        # Calculate lower and upper thresholds based on ht
        imin = np.min(image)
        imax = np.max(image)
        lower_threshold_count = int(ht)
        upper_threshold_count = int(total_pixels - ht)
        sorted_pixels = np.sort(image.flatten())
        imin = sorted_pixels[lower_threshold_count]
        imax = sorted_pixels[upper_threshold_count]

    # Apply histogram stretching to selected channels
    stretched_image = image.copy()
    for channel in channels:
        for i in range(height):
            for j in range(width):
                old_value = stretched_image[i, j, channel]
                if old_value < imin:
                    stretched_image[i, j, channel] = 0
                elif old_value > imax:
                    stretched_image[i, j, channel] = 255
                else:
                    stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)
    
    return stretched_image

# Example usage:
stretched_image_model = histogram_stretching(restored_image, average_red, r_threshold)




In [103]:
cv2.imwrite('stretched_image.jpg', stretched_image)

True

In [104]:
cv2.imwrite('stretched_image_model.jpg', stretched_image_model)

True